# Chapter 08: 공급망 에이전트 (Supply Chain Agent)

이 노트북에서는 공급망/물류 도메인의 멀티 에이전트 설계를 다룹니다.

## 주요 내용
- 공급망 물류 에이전트
- Actor-Critic 및 멀티 에이전트
- ADAS/A2A 설계

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/TeeDDub/Building-Applications-with-AI-Agents/blob/main/notebook/ch08_supply_chain_agent.ipynb)


## 1. 패키지 설치


In [ ]:
!pip install -q langchain langchain-openai langgraph openai numpy pandas ray backoff tqdm requests python-dotenv traceloop-sdk


## 2. API 키 설정


In [ ]:
import os

try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("✅ Colab Secrets에서 API 키를 불러왔습니다.")
except:
    pass

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = "sk-your-api-key-here"
    print("⚠️ API 키를 직접 입력해주세요.")


## 3. supply_chain_logistics_agent.py


공급망 물류 에이전트 워크플로우를 정의합니다.


In [ ]:
from __future__ import annotations
"""
supply_chain_logistics_agent.py
재고 관리, 운송 작업, 공급업체 관계 및 창고 최적화를 처리하는 
공급망 및 물류 관리 에이전트를 위한 LangGraph 워크플로우.
"""
import os
import json
import operator
import builtins
from typing import Annotated, Sequence, TypedDict, Optional

from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, SystemMessage
from langchain_core.messages.tool import ToolMessage
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain_core.tools import tool
from langgraph.graph import StateGraph, END

from traceloop.sdk import Traceloop
from src.common.observability.loki_logger import log_to_loki

# 환경변수
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "http://localhost:4317"
os.environ["OTEL_EXPORTER_OTLP_INSECURE"] = "true"

@tool
def manage_inventory(sku: Optional[str] = None, **kwargs) -> str:
    """재고 수준, 재고 보충, 감사 및 최적화 전략을 관리합니다."""
    print(f"[도구] manage_inventory(sku={sku}, kwargs={kwargs})")
    log_to_loki("tool.manage_inventory", f"sku={sku}")
    return "inventory_management_initiated"

@tool
def track_shipments(origin: Optional[str] = None, **kwargs) -> str:
    """배송 상태, 지연 사항을 추적하고 배송 물류를 조정합니다."""
    print(f"[도구] track_shipments(origin={origin}, kwargs={kwargs})")
    log_to_loki("tool.track_shipments", f"origin={origin}")
    return "shipment_tracking_updated"

@tool
def evaluate_suppliers(supplier_name: Optional[str] = None, **kwargs) -> str:
    """공급업체 성과를 평가하고 감사를 수행하며 공급업체 관계를 관리합니다."""
    print(f"[도구] evaluate_suppliers(supplier_name={supplier_name}, kwargs={kwargs})")
    log_to_loki("tool.evaluate_suppliers", f"supplier_name={supplier_name}")
    return "supplier_evaluation_complete"

@tool
def optimize_warehouse(operation_type: Optional[str] = None, **kwargs) -> str:
    """창고 운영, 레이아웃, 용량 및 보관 효율성을 최적화합니다."""
    print(f"[도구] optimize_warehouse(operation_type={operation_type}, kwargs={kwargs})")
    log_to_loki("tool.optimize_warehouse", f"operation_type={operation_type}")
    return "warehouse_optimization_initiated"

@tool
def forecast_demand(season: Optional[str] = None, **kwargs) -> str:
    """수요 패턴, 계절적 추세를 분석하고 예측 모델을 생성합니다."""
    print(f"[도구] forecast_demand(season={season}, kwargs={kwargs})")
    log_to_loki("tool.forecast_demand", f"season={season}")
    return "demand_forecast_generated"

@tool
def manage_quality(supplier: Optional[str] = None, **kwargs) -> str:
    """품질 관리, 결함 추적 및 공급업체 품질 표준을 관리합니다."""
    print(f"[도구] manage_quality(supplier={supplier}, kwargs={kwargs})")
    log_to_loki("tool.manage_quality", f"supplier={supplier}")
    return "quality_management_initiated"

@tool
def arrange_shipping(shipping_type: Optional[str] = None, **kwargs) -> str:
    """배송 방법, 특급 배송 및 복합 운송을 준비합니다."""
    print(f"[도구] arrange_shipping(shipping_type={shipping_type}, kwargs={kwargs})")
    log_to_loki("tool.arrange_shipping", f"shipping_type={shipping_type}")
    return "shipping_arranged"

@tool
def coordinate_operations(operation_type: Optional[str] = None, **kwargs) -> str:
    """크로스도킹, 통합 및 이동과 같은 복잡한 작업을 조정합니다."""
    print(f"[도구] coordinate_operations(operation_type={operation_type}, kwargs={kwargs})")
    log_to_loki("tool.coordinate_operations", f"operation_type={operation_type}")
    return "operations_coordinated"

@tool
def manage_special_handling(product_type: Optional[str] = None, **kwargs) -> str:
    """위험물, 콜드체인 및 민감한 제품에 대한 특수 요구사항을 처리합니다."""
    print(f"[도구] manage_special_handling(product_type={product_type}, kwargs={kwargs})")
    log_to_loki("tool.manage_special_handling", f"product_type={product_type}")
    return "special_handling_managed"

@tool
def handle_compliance(compliance_type: Optional[str] = None, **kwargs) -> str:
    """규제 준수, 세관, 문서화 및 인증을 관리합니다."""
    print(f"[도구] handle_compliance(compliance_type={compliance_type}, kwargs={kwargs})")
    log_to_loki("tool.handle_compliance", f"compliance_type={compliance_type}")
    return "compliance_handled"

@tool
def process_returns(returned_quantity: Optional[str] = None, **kwargs) -> str:
    """반품, 역물류 및 제품 처리를 처리합니다."""
    print(f"[도구] process_returns(returned_quantity={returned_quantity}, kwargs={kwargs})")
    log_to_loki("tool.process_returns", f"returned_quantity={returned_quantity}")
    return "returns_processed"

@tool
def scale_operations(scaling_type: Optional[str] = None, **kwargs) -> str:
    """성수기, 용량 계획 및 인력 관리를 위한 운영을 확장합니다."""
    print(f"[도구] scale_operations(scaling_type={scaling_type}, kwargs={kwargs})")
    log_to_loki("tool.scale_operations", f"scaling_type={scaling_type}")
    return "operations_scaled"

@tool
def optimize_costs(cost_type: Optional[str] = None, **kwargs) -> str:
    """운송, 보관 및 운영 비용을 분석하고 최적화합니다."""
    print(f"[도구] optimize_costs(cost_type={cost_type}, kwargs={kwargs})")
    log_to_loki("tool.optimize_costs", f"cost_type={cost_type}")
    return "cost_optimization_initiated"

@tool
def optimize_delivery(delivery_type: Optional[str] = None, **kwargs) -> str:
    """배송 경로, 라스트마일 물류 및 지속가능성 이니셔티브를 최적화합니다."""
    print(f"[도구] optimize_delivery(delivery_type={delivery_type}, kwargs={kwargs})")
    log_to_loki("tool.optimize_delivery", f"delivery_type={delivery_type}")
    return "delivery_optimization_complete"

@tool
def manage_disruption(disruption_type: Optional[str] = None, **kwargs) -> str:
    """공급망 중단, 비상 계획 및 위험 완화를 관리합니다."""
    print(f"[도구] manage_disruption(disruption_type={disruption_type}, kwargs={kwargs})")
    log_to_loki("tool.manage_disruption", f"disruption_type={disruption_type}")
    return "disruption_managed"

@tool
def send_logistics_response(operation_id: Optional[str] = None, message: Optional[str] = None) -> str:
    """이해관계자에게 물류 업데이트, 권장 사항 또는 상태 보고서를 전송합니다."""
    print(f"[도구] send_logistics_response → {message}")
    log_to_loki("tool.send_logistics_response", f"operation_id={operation_id}, message={message}")
    return "logistics_response_sent"

TOOLS = [
    manage_inventory, track_shipments, evaluate_suppliers, optimize_warehouse,
    forecast_demand, manage_quality, arrange_shipping, coordinate_operations,
    manage_special_handling, handle_compliance, process_returns, scale_operations,
    optimize_costs, optimize_delivery, manage_disruption, send_logistics_response
]


llm = init_chat_model(model="gpt-5-mini", callbacks=[StreamingStdOutCallbackHandler()],  
    verbose=True).bind_tools(TOOLS)

class AgentState(TypedDict):
    operation: Optional[dict]  # 공급망 운영 정보
    messages: Annotated[Sequence[BaseMessage], operator.add]

def call_model(state: AgentState):
    history = state["messages"]
    
    # 누락되거나 불완전한 작업 데이터를 적절히 처리
    operation = state.get("operation", {})
    if not operation:
        operation = {"operation_id": "UNKNOWN", "type": "general", "priority": "medium", "status": "active"}
    
    operation_json = json.dumps(operation, ensure_ascii=False)
    system_prompt = f"""
        당신은 숙련된 공급망 및 물류 관리 전문가입니다.
        전문 분야:
        - 재고 관리 및 수요 예측
        - 운송 및 배송 최적화
        - 공급업체 관계 관리 및 평가
        - 창고 운영 및 용량 계획
        - 품질 관리 및 규정 준수 관리
        - 비용 최적화 및 운영 효율성
        - 위험 관리 및 중단 대응
        - 지속가능성 및 친환경 물류 이니셔티브

        공급망 운영을 관리할 때:
        1) 물류 과제 또는 기회를 분석합니다
        2) 적절한 공급망 관리 도구를 호출합니다
        3) send_logistics_response로 권장 사항을 제공합니다
        4) 비용, 효율성, 품질 및 지속가능성 영향을 고려합니다
        5) 고객 만족도와 비즈니스 연속성을 우선시합니다

        항상 비용 최적화와 서비스 품질 및 위험 완화의 균형을 유지하십시오.

        작업: {operation_json}"""

    full = [SystemMessage(content=system_prompt)] + history

    first: ToolMessage | BaseMessage = llm.invoke(full)
    messages = [first]

    if getattr(first, "tool_calls", None):
        for tc in first.tool_calls:
            print(first)
            print(tc['name'])
            fn = next(t for t in TOOLS if t.name == tc['name'])
            out = fn.invoke(tc["args"])
            messages.append(ToolMessage(content=str(out), tool_call_id=tc["id"]))

        second = llm.invoke(full + messages)
        messages.append(second)

    return {"messages": messages}

def construct_graph():
    g = StateGraph(AgentState)
    g.add_node("assistant", call_model)
    g.set_entry_point("assistant")
    return g.compile()

graph = construct_graph()

if __name__ == "__main__":
    Traceloop.init(disable_batch=True, app_name="supply_chain_logistics_agent_langgraph")
    example = {"operation_id": "OP-12345", "type": "inventory_management", "priority": "high", "location": "Warehouse A"}
    convo = [HumanMessage(content="SKU-12345 재고가 심각하게 부족합니다. 현재 재고는 50개이지만 미처리 주문이 200개입니다. 재주문 전략은 무엇입니까?")]
    result = graph.invoke({"operation": example, "messages": convo})
    for m in result["messages"]:
        print(f"{m.type}: {m.content}") 


## 4. supply_chain_logistics_actor_critic.py


Actor-Critic 기반 공급망 멀티 에이전트를 구현합니다.


In [ ]:
from __future__ import annotations
"""
supply_chain_logistics_actor_critic.py
Actor-Critic 패턴을 적용한 멀티 에이전트 공급망 및 물류 관리 시스템.
Actor가 여러 후보 계획을 생성하고, Critic이 평가하여 최적의 계획을 선택하거나 재생성을 요청합니다.
"""
import os
import json
import operator
from typing import Annotated, Sequence, TypedDict, Optional

from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, SystemMessage
from langchain_core.messages.tool import ToolMessage
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain_core.tools import tool
from langgraph.graph import StateGraph, END

from traceloop.sdk import Traceloop
from src.common.observability.loki_logger import log_to_loki

# 환경변수
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "http://localhost:4317"
os.environ["OTEL_EXPORTER_OTLP_INSECURE"] = "true"

# 모든 전문가를 위한 공유 도구
@tool
def send_logistics_response(operation_id: Optional[str] = None, message: Optional[str] = None) -> str:
    """이해관계자에게 물류 업데이트, 권장 사항 또는 상태 보고서를 전송합니다."""
    print(f"[도구] send_logistics_response → {message}")
    log_to_loki("tool.send_logistics_response", f"operation_id={operation_id}, message={message}")
    return "logistics_response_sent"

# 재고 및 창고 전문가 도구
@tool
def manage_inventory(sku: Optional[str] = None, **kwargs) -> str:
    """재고 수준, 재고 보충, 감사 및 최적화 전략을 관리합니다."""
    print(f"[도구] manage_inventory(sku={sku}, kwargs={kwargs})")
    log_to_loki("tool.manage_inventory", f"sku={sku}")
    return "inventory_management_initiated"

@tool
def optimize_warehouse(operation_type: Optional[str] = None, **kwargs) -> str:
    """창고 운영, 레이아웃, 용량 및 보관 효율성을 최적화합니다."""
    print(f"[도구] optimize_warehouse(operation_type={operation_type}, kwargs={kwargs})")
    log_to_loki("tool.optimize_warehouse", f"operation_type={operation_type}")
    return "warehouse_optimization_initiated"

@tool
def forecast_demand(season: Optional[str] = None, **kwargs) -> str:
    """수요 패턴, 계절적 추세를 분석하고 예측 모델을 생성합니다."""
    print(f"[도구] forecast_demand(season={season}, kwargs={kwargs})")
    log_to_loki("tool.forecast_demand", f"season={season}")
    return "demand_forecast_generated"

@tool
def manage_quality(supplier: Optional[str] = None, **kwargs) -> str:
    """품질 관리, 결함 추적 및 공급업체 품질 표준을 관리합니다."""
    print(f"[도구] manage_quality(supplier={supplier}, kwargs={kwargs})")
    log_to_loki("tool.manage_quality", f"supplier={supplier}")
    return "quality_management_initiated"

@tool
def scale_operations(scaling_type: Optional[str] = None, **kwargs) -> str:
    """성수기, 용량 계획 및 인력 관리를 위한 운영을 확장합니다."""
    print(f"[도구] scale_operations(scaling_type={scaling_type}, kwargs={kwargs})")
    log_to_loki("tool.scale_operations", f"scaling_type={scaling_type}")
    return "operations_scaled"

@tool
def optimize_costs(cost_type: Optional[str] = None, **kwargs) -> str:
    """운송, 보관 및 운영 비용을 분석하고 최적화합니다."""
    print(f"[도구] optimize_costs(cost_type={cost_type}, kwargs={kwargs})")
    log_to_loki("tool.optimize_costs", f"cost_type={cost_type}")
    return "cost_optimization_initiated"

INVENTORY_TOOLS = [manage_inventory, optimize_warehouse, forecast_demand, manage_quality, scale_operations, optimize_costs, send_logistics_response]

# 운송 및 물류 전문가 도구
@tool
def track_shipments(origin: Optional[str] = None, **kwargs) -> str:
    """배송 상태, 지연 사항을 추적하고 배송 물류를 조정합니다."""
    print(f"[도구] track_shipments(origin={origin}, kwargs={kwargs})")
    log_to_loki("tool.track_shipments", f"origin={origin}")
    return "shipment_tracking_updated"

@tool
def arrange_shipping(shipping_type: Optional[str] = None, **kwargs) -> str:
    """배송 방법, 특급 배송 및 복합 운송을 준비합니다."""
    print(f"[도구] arrange_shipping(shipping_type={shipping_type}, kwargs={kwargs})")
    log_to_loki("tool.arrange_shipping", f"shipping_type={shipping_type}")
    return "shipping_arranged"

@tool
def coordinate_operations(operation_type: Optional[str] = None, **kwargs) -> str:
    """크로스도킹, 통합 및 이동과 같은 복잡한 작업을 조정합니다."""
    print(f"[도구] coordinate_operations(operation_type={operation_type}, kwargs={kwargs})")
    log_to_loki("tool.coordinate_operations", f"operation_type={operation_type}")
    return "operations_coordinated"

@tool
def manage_special_handling(product_type: Optional[str] = None, **kwargs) -> str:
    """위험물, 콜드체인 및 민감한 제품에 대한 특수 요구사항을 처리합니다."""
    print(f"[도구] manage_special_handling(product_type={product_type}, kwargs={kwargs})")
    log_to_loki("tool.manage_special_handling", f"product_type={product_type}")
    return "special_handling_managed"

@tool
def process_returns(returned_quantity: Optional[str] = None, **kwargs) -> str:
    """반품, 역물류 및 제품 처리를 처리합니다."""
    print(f"[도구] process_returns(returned_quantity={returned_quantity}, kwargs={kwargs})")
    log_to_loki("tool.process_returns", f"returned_quantity={returned_quantity}")
    return "returns_processed"

@tool
def optimize_delivery(delivery_type: Optional[str] = None, **kwargs) -> str:
    """배송 경로, 라스트마일 물류 및 지속가능성 이니셔티브를 최적화합니다."""
    print(f"[도구] optimize_delivery(delivery_type={delivery_type}, kwargs={kwargs})")
    log_to_loki("tool.optimize_delivery", f"delivery_type={delivery_type}")
    return "delivery_optimization_complete"

@tool
def manage_disruption(disruption_type: Optional[str] = None, **kwargs) -> str:
    """공급망 중단, 비상 계획 및 위험 완화를 관리합니다."""
    print(f"[도구] manage_disruption(disruption_type={disruption_type}, kwargs={kwargs})")
    log_to_loki("tool.manage_disruption", f"disruption_type={disruption_type}")
    return "disruption_managed"

TRANSPORTATION_TOOLS = [track_shipments, arrange_shipping, coordinate_operations, manage_special_handling, process_returns, optimize_delivery, manage_disruption, send_logistics_response]

# 공급업체 및 규정 준수 전문가 도구
@tool
def evaluate_suppliers(supplier_name: Optional[str] = None, **kwargs) -> str:
    """공급업체 성과를 평가하고 감사를 수행하며 공급업체 관계를 관리합니다."""
    print(f"[도구] evaluate_suppliers(supplier_name={supplier_name}, kwargs={kwargs})")
    log_to_loki("tool.evaluate_suppliers", f"supplier_name={supplier_name}")
    return "supplier_evaluation_complete"

@tool
def handle_compliance(compliance_type: Optional[str] = None, **kwargs) -> str:
    """규제 준수, 세관, 문서화 및 인증을 관리합니다."""
    print(f"[도구] handle_compliance(compliance_type={compliance_type}, kwargs={kwargs})")
    log_to_loki("tool.handle_compliance", f"compliance_type={compliance_type}")
    return "compliance_handled"

SUPPLIER_TOOLS = [evaluate_suppliers, handle_compliance, send_logistics_response]

# 모든 도구 리스트
ALL_TOOLS = INVENTORY_TOOLS + TRANSPORTATION_TOOLS + SUPPLIER_TOOLS

llm = init_chat_model(model="gpt-5-mini", temperature=0.0, callbacks=[StreamingStdOutCallbackHandler()], verbose=True)

# AgentState 정의 - candidates와 iteration 필드 추가
class AgentState(TypedDict):
    operation: Optional[dict]  # 공급망 운영 정보
    messages: Annotated[Sequence[BaseMessage], operator.add]
    candidates: Optional[list]  # Actor가 생성한 후보 계획들
    iteration: Optional[int]  # 반복 횟수

# Actor 노드: 후보 계획 생성
def actor_node(state: AgentState):
    """3개의 후보 공급망 계획을 생성합니다."""
    history = state["messages"]
    actor_prompt = '''3개의 공급망 후보 계획을 JSON 리스트 형식으로 생성하세요.
    형식: [{'plan': '계획 설명', 'tools': [{'tool': '도구명', 'args': {...}}]}]
    각 계획은 실행 가능한 구체적인 단계와 필요한 도구를 포함해야 합니다.'''
    response = llm.invoke([SystemMessage(content=actor_prompt)] + history)
    try:
        candidates = json.loads(response.content)
    except json.JSONDecodeError:
        # JSON 파싱 실패 시 기본 후보 제공
        candidates = [{"plan": "기본 계획", "tools": []}]
    return {"candidates": candidates, "messages": state["messages"]}

# Critic 노드: 평가 및 선택/반복
def critic_node(state: AgentState):
    """후보 계획들을 평가하고 최적의 계획을 선택하거나 재생성을 요청합니다."""
    candidates = state.get("candidates", [])
    history = state["messages"]
    
    critic_prompt = f'''다음 후보 계획들을 평가하세요: {candidates}
    
    실행 가능성(feasibility), 비용(cost), 위험도(risk) 기준으로 각각 1-10점으로 채점하세요.
    
    응답 형식 (JSON):
    {{
        "evaluations": [
            {{"plan_index": 0, "feasibility": 점수, "cost": 점수, "risk": 점수, "total": 총점}},
            ...
        ],
        "best_score": 최고점수,
        "selected": 선택된_계획_객체,
        "feedback": "개선을 위한 피드백 (점수가 8점 이하인 경우)"
    }}
    
    최고 점수가 8점 이상이면 해당 계획을 선택하고, 그렇지 않으면 재생성을 요청하세요.'''
    
    response = llm.invoke([SystemMessage(content=critic_prompt)] + history)
    
    try:
        evaluation = json.loads(response.content)
    except json.JSONDecodeError:
        # JSON 파싱 실패 시 첫 번째 후보 선택
        evaluation = {
            "best_score": 9,
            "selected": candidates[0] if candidates else {"plan": "기본 계획", "tools": []},
            "feedback": ""
        }
    
    if evaluation.get('best_score', 0) > 8:
        winning_plan = evaluation['selected']
        # 선택된 계획의 도구들을 실행
        messages = []
        for tool_info in winning_plan.get('tools', []):
            tool_name = tool_info.get('tool', '')
            tool_args = tool_info.get('args', {})
            tc = {'name': tool_name, 'args': tool_args, 'id': f'tool_{len(messages)}'}
            
            # 도구 찾기 및 실행
            try:
                fn = next(t for t in ALL_TOOLS if t.name == tool_name)
                out = fn.invoke(tool_args)
                messages.append(ToolMessage(content=str(out), tool_call_id=tc["id"]))
            except StopIteration:
                print(f"[경고] 도구를 찾을 수 없음: {tool_name}")
            except Exception as e:
                print(f"[오류] 도구 실행 실패: {tool_name}, {e}")
        
        # 최종 응답 전송
        send_logistics_response.invoke({"message": winning_plan.get('plan', '계획 실행 완료')})
        
        final_message = AIMessage(
            content=f"선택된 계획: {winning_plan.get('plan', '')} (점수: {evaluation.get('best_score', 0)})"
        )
        return {"messages": history + messages + [final_message]}
    else:
        # 반복: Actor에게 피드백 제공
        feedback_message = AIMessage(
            content=f"재생성 필요: 개선 사항 - {evaluation.get('feedback', '더 나은 계획이 필요합니다.')}"
        )
        return {"messages": history + [feedback_message]}

# Actor-Critic 그래프 구성
def construct_actor_critic_graph():
    """Actor-Critic 패턴을 사용한 공급망 관리 그래프를 구성합니다."""
    g = StateGraph(AgentState)
    g.add_node("actor", actor_node)
    g.add_node("critic", critic_node)
    
    g.set_entry_point("actor")
    g.add_edge("actor", "critic")
    
    # 승인되지 않은 경우 다시 Actor로 돌아감 (조건부)
    def should_continue(state: AgentState) -> str:
        """Critic이 재생성을 요청했는지 확인합니다."""
        if not state.get("messages"):
            return END
        last_message = state["messages"][-1]
        if hasattr(last_message, 'content') and "재생성" in last_message.content:
            return "actor"
        return END
    
    g.add_conditional_edges("critic", should_continue)
    
    return g.compile()

# 그래프 컴파일
graph = construct_actor_critic_graph()

if __name__ == "__main__":
    Traceloop.init(disable_batch=True, app_name="supply_chain_logistics_agent_langgraph")
    print("=" * 80)
    print("공급망 물류 Actor-Critic 시스템 시작")
    print("=" * 80)
    
    # 예제 작업
    example_operation = {
        "operation_id": "OP-12345",
        "type": "inventory_management",
        "priority": "high",
        "location": "Warehouse A",
        "issue": "critical_shortage"
    }
    
    initial_message = HumanMessage(
        content="SKU-12345 재고가 심각하게 부족합니다. 현재 재고는 50개이지만 미처리 주문이 200개입니다. "
                "재주문 전략과 단기 해결책을 제시해주세요."
    )
    
    # 그래프 실행
    result = graph.invoke({
        "operation": example_operation,
        "messages": [initial_message],
        "candidates": None,
        "iteration": 0
    })
    
    print("\n" + "=" * 80)
    print("최종 결과")
    print("=" * 80)
    
    for i, m in enumerate(result["messages"], 1):
        msg_type = m.type if hasattr(m, 'type') else type(m).__name__
        content = m.content if hasattr(m, 'content') else str(m)
        print(f"\n[{i}] {msg_type}:")
        print(f"  {content[:200]}{'...' if len(content) > 200 else ''}")
    
    print("\n" + "=" * 80)
    print("실행 완료")
    print("=" * 80)



## 5. automated_design_of_agentic_systems.py


ADAS 프레임워크로 에이전트 시스템을 탐색합니다.


In [ ]:
import argparse
import copy
import json
import os
import pickle
import random
from utils import random_id, bootstrap_confidence_interval
from collections import namedtuple
from concurrent.futures import ThreadPoolExecutor


import backoff
import numpy as np
import openai
import pandas
from tqdm import tqdm

# 환경변수
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass

client = openai.OpenAI()

# 작업에 맞는 prompt 가져오기
# 새로운 작업을 위해서는 get_init_archive, get_prompt, get_reflexion_prompt 함수를 포함한 새로운 prompt 모듈을 생성하세요.

Info = namedtuple('Info', ['name', 'author', 'content', 'iteration_idx'])

FORMAT_INST = lambda request_keys: f"""Reply EXACTLY with the following JSON format.\n{str(request_keys)}\nDO NOT MISS ANY REQUEST FIELDS and ensure that your response is a well-formed JSON object!\n"""
ROLE_DESC = lambda role: f"You are a {role}."
SYSTEM_MSG = ""

PRINT_LLM_DEBUG = False
SEARCHING_MODE = True


@backoff.on_exception(backoff.expo, openai.RateLimitError)
def get_json_response_from_gpt(
        msg,
        model,
        system_message,
        temperature=0.5
):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": msg},
        ],
        temperature=temperature, stop=None, response_format={"type": "json_object"}
    )
    content = response.choices[0].message.content
    json_dict = json.loads(content)
    assert json_dict is not None
    return json_dict


@backoff.on_exception(backoff.expo, openai.RateLimitError)
def get_json_response_from_gpt_reflect(
        msg_list,
        model,
        temperature=1
):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=msg_list,
            temperature=temperature, stop=None, response_format={"type": "json_object"}
        )
        content = response.choices[0].message.content
        
        # 디버깅: 응답 내용 확인
        if PRINT_LLM_DEBUG:
            print(f"[DEBUG] Model: {model}")
            print(f"[DEBUG] Response content: {content[:200] if content else 'EMPTY'}")
            print(f"[DEBUG] Finish reason: {response.choices[0].finish_reason}")
            print(f"[DEBUG] Usage: {response.usage}")
        
        if not content:
            raise ValueError(f"Empty response from model {model}. Finish reason: {response.choices[0].finish_reason}")
        
        json_dict = json.loads(content)
        assert json_dict is not None
        return json_dict
    except Exception as e:
        if PRINT_LLM_DEBUG:
            print(f"[DEBUG] Exception in get_json_response_from_gpt_reflect: {e}")
            print(f"[DEBUG] Messages length: {len(msg_list)}")
        raise


class LLMAgentBase:
    """
    LLM 에이전트의 기본 클래스, 다양한 출력 형식을 위해 설정 가능합니다.
    """
    def __init__(self, output_fields: list, agent_name: str,
                 role='helpful assistant', model='gpt-5-mini', temperature=0.5) -> None:
        self.output_fields = output_fields
        self.agent_name = agent_name
        self.role = role
        self.model = model
        self.temperature = temperature
        self.id = random_id()  # Assume random_id from utils

    def generate_prompt(self, input_infos, instruction, output_description) -> tuple:
        output_fields_and_description = {key: output_description.get(key, f"Your {key}.") for key in self.output_fields}
        system_prompt = ROLE_DESC(self.role) + "\n\n" + FORMAT_INST(output_fields_and_description)

        input_infos_text = ''
        for input_info in input_infos:
            if isinstance(input_info, Info):
                (field_name, author, content, iteration_idx) = input_info
                if author == self.__repr__():
                    author += ' (yourself)'
                if field_name == 'task':
                    input_infos_text += f'# Your Task:\n{content}\n\n'
                elif iteration_idx != -1:
                    input_infos_text += f'### {field_name} #{iteration_idx + 1} by {author}:\n{content}\n\n'
                else:
                    input_infos_text += f'### {field_name} by {author}:\n{content}\n\n'

        prompt = input_infos_text + instruction
        return system_prompt, prompt

    def query(self, input_infos: list, instruction, output_description, iteration_idx=-1) -> list:
        system_prompt, prompt = self.generate_prompt(input_infos, instruction, output_description)
        try:
            response_json = get_json_response_from_gpt(prompt, self.model, system_prompt, self.temperature)
            assert len(response_json) == len(self.output_fields), "not returning enough fields"
        except Exception as e:
            response_json = {key: '' for key in self.output_fields if key not in response_json}
            for key in list(response_json):
                if key not in self.output_fields:
                    del response_json[key]
        output_infos = [Info(key, self.__repr__(), value, iteration_idx) for key, value in response_json.items()]
        return output_infos

    def __repr__(self):
        return f"{self.agent_name} {self.id}"

    def __call__(self, input_infos: list, instruction, output_description, iteration_idx=-1):
        return self.query(input_infos, instruction, output_description, iteration_idx)


class AgentSystem:
    """
    AgentSystem의 기본 클래스, ARC에서 피드백과 같은 작업별 동작을 확장할 수 있습니다.
    """
    def __init__(self, **task_specific_init):
        for k, v in task_specific_init.items():
            setattr(self, k, v)


class BaseTask:
    """
    작업의 추상 기본 클래스, 새로운 문제를 위해 서브클래스를 생성하세요.
    Required methods:
    - get_init_archive: 초기 솔루션.
    - get_prompt: 새로운 솔루션을 생성하기 위한 프롬프트.
    - get_reflexion_prompt: 반영을 위한 프롬프트.
    - load_data: 검증/테스트 데이터 로드.
    - format_task: 데이터를 프롬프트 문자열로 포맷.
    - get_ground_truth: 데이터에서 진실 추출.
    - evaluate_prediction: 예측과 진실 간의 점수 매기기 (예: 1/0 정확도).
    - parse_prediction: 원시 전방 출력을 비교 가능한 형식으로 파싱.
    - get_output_description: 프롬프트의 출력 필드를 딕셔너리로 가져오기.
    - get_instruction: 프롬프트에 추가 지시사항.

    Optional:
    - prepare_task_queue: 병렬 평가를 위한 입력 준비 (기본: 간단한 리스트).
    - get_agent_system: 사용자 정의 AgentSystem 인스턴스 (기본: 기본).
    """
    def __init__(self, args):
        self.args = args

    def get_init_archive(self):
        raise NotImplementedError

    def get_prompt(self, archive):
        raise NotImplementedError

    def get_reflexion_prompt(self, prev_solution):
        raise NotImplementedError

    def load_data(self, mode):  # mode: True for search (val), False for eval (test)
        raise NotImplementedError

    def format_task(self, task_data):
        raise NotImplementedError

    def get_ground_truth(self, task_data):
        raise NotImplementedError

    def evaluate_prediction(self, prediction, ground_truth):
        raise NotImplementedError

    def parse_prediction(self, res):
        raise NotImplementedError

    def get_output_description(self):
        return {}

    def get_instruction(self):
        return ""

    def prepare_task_queue(self, data):
        return [Info('task', 'User', self.format_task(d), -1) for d in data]

    def get_agent_system(self, task_data=None):
        return AgentSystem()


# MMLU 예제 서브클래스 (이전 코드 기반)
class MMLUTask(BaseTask):
    def get_init_archive(self):
        from mmlu_prompt import get_init_archive  # Task-specific
        return get_init_archive()

    def get_prompt(self, archive):
        from mmlu_prompt import get_prompt
        return get_prompt(archive)

    def get_reflexion_prompt(self, prev_solution):
        from mmlu_prompt import get_reflexion_prompt
        return get_reflexion_prompt(prev_solution)

    def load_data(self, mode):
        df = pandas.read_csv(self.args.data_filename)
        random.seed(self.args.shuffle_seed)
        examples = [row.to_dict() for _, row in df.iterrows()]
        random.shuffle(examples)
        if mode:
            return examples[:self.args.valid_size] * self.args.n_repeat
        else:
            start = self.args.valid_size
            end = start + self.args.test_size
            return examples[start:end] * self.args.n_repeat

    def format_task(self, task_data):
        from utils import format_multichoice_question  # Assume in utils
        return format_multichoice_question(task_data)

    def get_ground_truth(self, task_data):
        LETTER_TO_INDEX = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
        return LETTER_TO_INDEX[task_data['Answer']]

    def evaluate_prediction(self, prediction, ground_truth):
        return 1 if prediction == ground_truth else 0

    def parse_prediction(self, res):
        LETTER_TO_INDEX = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
        try:
            if isinstance(res, str) and res in LETTER_TO_INDEX:
                return LETTER_TO_INDEX[res]
            if 'A)' in str(res):
                return 0
            if 'B)' in str(res):
                return 1
            if 'C)' in str(res):
                return 2
            if 'D)' in str(res):
                return 3
            if isinstance(res, list) and len(res) > 1:
                content = res[1].content
                return LETTER_TO_INDEX[content] if content in LETTER_TO_INDEX else -1
            if hasattr(res, 'content'):
                content = res.content
                if content in LETTER_TO_INDEX:
                    return LETTER_TO_INDEX[content]
                if 'A)' in content:
                    return 0
                if 'B)' in content:
                    return 1
                if 'C)' in content:
                    return 2
                if 'D)' in content:
                    return 3
        except:
            pass
        return -1

    def get_output_description(self):
        return {'answer': '당신의 답변. 알파벳 선택만 반환하세요, 즉 A 또는 B 또는 C 또는 D.'}


# ARC 예제 서브클래스 (이전 코드 기반)
class ARCTask(BaseTask):
    def get_init_archive(self):
        from arc_prompt import get_init_archive
        return get_init_archive()

    def get_prompt(self, archive):
        from arc_prompt import get_prompt
        return get_prompt(archive)

    def get_reflexion_prompt(self, prev_solution):
        from arc_prompt import get_reflexion_prompt
        return get_reflexion_prompt(prev_solution)

    def load_data(self, mode):
        path = self.args.val_data_path if mode else self.args.test_data_path
        with open(path, 'rb') as f:
            data = pickle.load(f)
        return data * self.args.n_repeat

    def format_task(self, task_data):
        from utils import format_arc_data
        task_str, _, _ = format_arc_data(task_data)
        return task_str

    def get_ground_truth(self, task_data):
        return task_data['test'][0]['output']  # ARC 구조

    def evaluate_prediction(self, prediction, ground_truth):
        from utils import eval_solution
        arc_data = {'test': [{'output': ground_truth}]}
        return eval_solution(prediction, arc_data, soft_eval=False)

    def parse_prediction(self, res):
        try:
            if isinstance(res, Info):
                res = res.content
            if isinstance(res, str):
                res = eval(res)
            return res
        except:
            return None

    def get_output_description(self):
        return {'answer': '당신의 답변. list[list[int]] 형식의 문자열만 반환하세요. 다른 것은 반환하지 마세요.'}

    def get_instruction(self):
        return "`transform` 함수를 생성하여 이 작업을 해결하세요. 이 함수는 단일 인수를 받아들이며, 입력 그리드를 `list[list[int]]` 형식으로 받아들이고, 변환된 그리드를 `list[list[int]]` 형식으로 반환해야 합니다."

    def prepare_task_queue(self, data):
        from utils import format_arc_data
        queue = []
        for arc_data in data:
            task_str, examples, test_input = format_arc_data(arc_data)
            taskInfo = Info('task', 'User', task_str, -1)
            agent = self.get_agent_system(examples=examples, test_input=test_input)
            queue.append((agent, taskInfo, arc_data))
        return queue

    def get_agent_system(self, **kwargs):
        from utils import list_to_string  # Assume in utils
        class ARCAgentSystem(AgentSystem):
            def __init__(self, examples, test_input):
                super().__init__(examples=examples, test_input=test_input)

            # run_examples_and_get_feedback 및 get_test_output_from_code가 필요한 경우 추가
            # (omitted for brevity, but can copy from original ARC code)

        return ARCAgentSystem(kwargs['examples'], kwargs['test_input'])


def search(args, task):
    file_path = os.path.join(args.save_dir, f"{args.expr_name}_run_archive.json")
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            archive = json.load(f)
        last_gen = archive[-1].get('generation', 0) if archive else 0
        start = int(last_gen) if isinstance(last_gen, int) else 0
    else:
        archive = task.get_init_archive()
        start = 0

    # 초기 아카이브 평가
    for solution in archive:
        if 'fitness' in solution:
            continue
        solution['generation'] = "initial"
        print(f"============Initial Archive: {solution.get('name', 'unnamed')}=================")
        try:
            acc_list = evaluate_forward_fn(args, solution["code"], task)
        except Exception as e:
            print(f"Error evaluating initial: {e}")
            continue
        solution['fitness'] = bootstrap_confidence_interval(acc_list)  
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, 'w') as f:
            json.dump(archive, f, indent=4)

    for n in range(start, args.n_generation):
        print(f"============Generation {n + 1}=================")
        system_prompt, prompt = task.get_prompt(archive)
        msg_list = [{"role": "system", "content": system_prompt}, {"role": "user", "content": prompt}]
        try:
            next_solution = get_json_response_from_gpt_reflect(msg_list, args.model)
            ref1, ref2 = task.get_reflexion_prompt(archive[-1] if n > 0 else None)
            msg_list += [{"role": "assistant", "content": str(next_solution)}, {"role": "user", "content": ref1}]
            next_solution = get_json_response_from_gpt_reflect(msg_list, args.model)
            msg_list += [{"role": "assistant", "content": str(next_solution)}, {"role": "user", "content": ref2}]
            next_solution = get_json_response_from_gpt_reflect(msg_list, args.model)
        except Exception as e:
            print(f"Error generating solution: {e}")
            continue

        acc_list = []
        for _ in range(args.debug_max):
            try:
                acc_list = evaluate_forward_fn(args, next_solution["code"], task)
                if np.mean(acc_list) < 0.01 and SEARCHING_MODE:
                    raise Exception("All 0 accuracy")
                break
            except Exception as e:
                print(f"Evaluation error: {e}")
                msg_list += [{"role": "assistant", "content": str(next_solution)}, 
                             {"role": "user", "content": f"Error: {e}\nDebug and repeat thought in 'thought', debug in 'debug_thought'"}]
                try:
                    next_solution = get_json_response_from_gpt_reflect(msg_list, args.model)
                except Exception as ee:
                    print(f"Error debugging: {ee}")
                    break
        if not acc_list:
            continue

        next_solution['fitness'] = bootstrap_confidence_interval(acc_list)
        next_solution['generation'] = n + 1
        next_solution.pop('debug_thought', None)
        next_solution.pop('reflection', None)
        archive.append(next_solution)

        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, 'w') as f:
            json.dump(archive, f, indent=4)


def evaluate(args, task):
    file_path = os.path.join(args.save_dir, f"{args.expr_name}_run_archive.json")
    eval_file_path = file_path.replace(".json", "_evaluate.json")
    with open(file_path, 'r') as f:
        archive = json.load(f)
    eval_archive = json.load(open(eval_file_path)) if os.path.exists(eval_file_path) else []

    current_idx = len(eval_archive)
    while current_idx < len(archive):
        sol = archive[current_idx]
        print(f"Evaluating gen {sol['generation']}, idx {current_idx}")
        try:
            acc_list = evaluate_forward_fn(args, sol["code"], task)
            sol['test_fitness'] = bootstrap_confidence_interval(acc_list)
            eval_archive.append(sol)
            with open(eval_file_path, 'w') as f:
                json.dump(eval_archive, f, indent=4)
        except Exception as e:
            print(f"Error: {e}")
        current_idx += 1


def evaluate_forward_fn(args, forward_str, task):
    namespace = {}
    exec(forward_str, globals(), namespace)
    func = list(namespace.values())[0]
    if not callable(func):
        raise ValueError("Not callable")
    AgentSystem.forward = staticmethod(func)  # 클래스에 첨부

    data = task.load_data(SEARCHING_MODE)
    task_queue = task.prepare_task_queue(data)
    max_workers = min(len(task_queue), args.max_workers) if args.multiprocessing else 1

    def process_item(item):
        # ARC task는 (agent, taskInfo, task_data) tuple을 반환하고
        # MMLU task는 Info 객체만 반환합니다.
        # Info는 namedtuple이므로 isinstance(item, tuple)이 True지만
        # 길이가 3이 아니라 4입니다.
        if isinstance(item, tuple) and not isinstance(item, Info):
            agent, taskInfo, task_data = item
        else:
            agent = task.get_agent_system()
            taskInfo = item
            # 해당 데이터 찾기 (인덱스 필요 시)
            idx = task_queue.index(item)
            task_data = data[idx]
        res = agent.forward(taskInfo)
        prediction = task.parse_prediction(res)
        ground_truth = task.get_ground_truth(task_data)
        if prediction is None:
            return 0
        return task.evaluate_prediction(prediction, ground_truth)

    with ThreadPoolExecutor(max_workers) as executor:
        acc_list = list(tqdm(executor.map(process_item, task_queue), total=len(task_queue)))

    print(f"acc: {bootstrap_confidence_interval(acc_list)}")  
    return acc_list


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Generic ADAS Framework")
    parser.add_argument('--task', type=str, required=True, choices=['mmlu', 'arc'], help="Task type")
    # Common
    parser.add_argument('--save_dir', type=str, default='results/')
    parser.add_argument('--expr_name', type=str, default='adas')
    parser.add_argument('--n_generation', type=int, default=30)
    parser.add_argument('--debug_max', type=int, default=3)
    parser.add_argument('--model', type=str, default='gpt-5-mini')
    parser.add_argument('--n_repeat', type=int, default=1)
    parser.add_argument('--multiprocessing', action='store_true')
    parser.add_argument('--max_workers', type=int, default=48)
    # MMLU-specific
    parser.add_argument('--data_filename', type=str)
    parser.add_argument('--valid_size', type=int, default=128)
    parser.add_argument('--test_size', type=int, default=800)
    parser.add_argument('--shuffle_seed', type=int, default=0)
    # ARC-specific
    parser.add_argument('--val_data_path', type=str)
    parser.add_argument('--test_data_path', type=str)

    args = parser.parse_args()

    if args.task == 'mmlu':
        task = MMLUTask(args)
        args.expr_name += '_mmlu'
    elif args.task == 'arc':
        if not args.val_data_path or not args.test_data_path:
            print("오류: ARC 태스크는 --val_data_path와 --test_data_path 인자가 필요합니다.")
            exit(1)
        task = ARCTask(args)
        args.expr_name += '_arc'
    else:
        print("Error: Invalid task type. Please choose 'mmlu' or 'arc'.")
        exit(1)

    SEARCHING_MODE = True
    search(args, task)


## 6. agent_server.py


이 코드는 HTTP 서버를 실행합니다. Colab에서는 서버 실행이 제한될 수 있으니 원본 파일을 참고해 로컬에서 실행하세요.


A2A 스펙 기반 에이전트 서버를 구현합니다.


In [ ]:
import json
import os
import uuid
from http.server import BaseHTTPRequestHandler, HTTPServer
from openai import OpenAI

# 환경변수 로드
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass

# 에이전트 카드 (발견을 위한 JSON 설명자)
agent_card = {
    "name": "SummarizerAgent",
    "description": "텍스트 요약을 수행하는 AI 에이전트입니다.",
    "protocolVersion": "1.0",
    "url": "http://localhost:8000",
    "provider": {
        "organization": "Example Org",
        "url": "https://example.org"
    },
    "capabilities": {
        "streaming": False,
        "pushNotifications": False,
        "stateTransitionHistory": False
    },
    "skills": [
        {
            "id": "summarize-text",
            "name": "텍스트 요약",
            "description": "주어진 텍스트를 간결하게 요약합니다.",
            "tags": ["summarization", "nlp", "text-processing"],
            "examples": [
                "이 기사를 요약해 주세요",
                "다음 내용을 간단히 정리해 주세요"
            ]
        }
    ],
    "defaultInputModes": ["text/plain"],
    "defaultOutputModes": ["text/plain"],
    "security": []  # 프로덕션에서는 OAuth2, API 키 등을 설정하세요
}


class AgentHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        # A2A 스펙: /.well-known/agent-card.json (섹션 8.2, 14.3)
        if self.path == '/.well-known/agent-card.json':
            self.send_response(200)
            self.send_header('Content-type', 'application/json; charset=utf-8')
            self.end_headers()
            self.wfile.write(json.dumps(agent_card, ensure_ascii=False).encode('utf-8'))
        else:
            self.send_response(404)
            self.end_headers()

    def do_POST(self):
        if self.path == '/':
            content_length = int(self.headers['Content-Length'])
            post_data = self.rfile.read(content_length)
            rpc_request = json.loads(post_data)
            
            # A2A JSON-RPC 요청 처리 (섹션 9.4.1 message/send)
            if rpc_request.get('jsonrpc') == '2.0' and rpc_request['method'] == 'message/send':
                params = rpc_request.get('params', {})
                message = params.get('message', {})
                parts = message.get('parts', [])
                
                # 텍스트 파트 추출 (섹션 4.1.6 Part)
                text = ""
                for part in parts:
                    if 'text' in part:
                        text += part['text']
                
                # OpenAI API를 사용한 실제 LLM 요약
                client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
                try:
                    llm_response = client.chat.completions.create(
                        model="gpt-4o-mini",
                        messages=[
                            {"role": "system", "content": "당신은 간결한 요약을 제공하는 유용한 어시스턴트입니다."},
                            {"role": "user", "content": f"다음 텍스트를 요약하세요:\n{text}"}
                        ],
                    )
                    summary = llm_response.choices[0].message.content.strip()
                except Exception as e:
                    summary = f"요약 중 오류 발생: {str(e)}"
                
                # A2A 스펙 준수 응답 (섹션 4.1.1 Task, 4.1.2 TaskStatus)
                task_id = str(uuid.uuid4())
                response = {
                    "jsonrpc": "2.0",
                    "result": {
                        "id": task_id,
                        "contextId": params.get('contextId', str(uuid.uuid4())),
                        "status": {
                            "state": "completed"
                        },
                        "artifacts": [
                            {
                                "parts": [{"text": summary}]
                            }
                        ]
                    },
                    "id": rpc_request['id']
                }
                self.send_response(200)
                self.send_header('Content-type', 'application/json; charset=utf-8')
                self.end_headers()
                self.wfile.write(json.dumps(response, ensure_ascii=False).encode('utf-8'))
            else:
                # JSON-RPC 오류 처리 (섹션 9.5)
                error_response = {
                    "jsonrpc": "2.0",
                    "error": {"code": -32601, "message": "Method not found"},
                    "id": rpc_request.get('id')
                }
                self.send_response(200)  # JSON-RPC 에러도 HTTP 200으로 반환
                self.send_header('Content-type', 'application/json; charset=utf-8')
                self.end_headers()
                self.wfile.write(json.dumps(error_response, ensure_ascii=False).encode('utf-8'))
        else:
            self.send_response(404)
            self.end_headers()


if __name__ == '__main__':
    server_address = ('', 8000)
    httpd = HTTPServer(server_address, AgentHandler)
    print("A2A 에이전트 서버를 시작합니다. 주소: http://localhost:8000")
    print("Agent Card: http://localhost:8000/.well-known/agent-card.json")
    httpd.serve_forever()


## 7. agent_client.py


이 코드는 로컬에서 실행 중인 A2A 서버에 연결합니다. Colab에서는 로컬 서버에 접속하기 어려우니 원본 파일을 참고해 로컬에서 실행하세요.


A2A 에이전트 호출 클라이언트를 구현합니다.


In [ ]:
import requests
import json
import uuid

# 단계 1: 에이전트 검색 - A2A 스펙 준수 well-known URI
card_url = 'http://localhost:8000/.well-known/agent-card.json'
response = requests.get(card_url)
if response.status_code != 200:
    print("에이전트 카드 가져오기 실패")
    exit()

agent_card = response.json()
print("발견된 에이전트 카드:", json.dumps(agent_card, indent=2, ensure_ascii=False))

# 단계 2: 핸드셰이크 (버전 및 기능 확인)
if agent_card.get('protocolVersion', '').split('.')[0] != '1':
    print("호환되지 않는 프로토콜 버전")
    exit()

# skills 확인
skills = agent_card.get('skills', [])
skill_ids = [s.get('id') for s in skills]
if "summarize-text" not in skill_ids:
    print("필요한 스킬이 지원되지 않음")
    exit()
print("핸드셰이크 성공: 에이전트가 호환됩니다.")

# 단계 3: A2A 스펙 준수 JSON-RPC 요청 (message/send)
rpc_url = agent_card['url']  # 에이전트 기본 URL로 POST
rpc_request = {
    "jsonrpc": "2.0",
    "method": "message/send",
    "params": {
        "contextId": str(uuid.uuid4()),
        "message": {
            "role": "user",
            "parts": [
                {
                    "text": "이것은 요약이 필요한 긴 예제 텍스트입니다. 멀티 에이전트 시스템, 통신 프로토콜, A2A와 같은 표준을 사용하여 에이전트들이 자율적으로 협업하는 방법을 논의합니다."
                }
            ]
        }
    },
    "id": 123  # 고유한 요청 ID
}

response = requests.post(rpc_url, json=rpc_request)
if response.status_code == 200:
    rpc_response = response.json()
    print("\nRPC 응답:", json.dumps(rpc_response, indent=2, ensure_ascii=False))
    
    # 결과 파싱
    if 'result' in rpc_response:
        result = rpc_response['result']
        print(f"\n태스크 ID: {result.get('id')}")
        print(f"상태: {result.get('status', {}).get('state')}")
        
        artifacts = result.get('artifacts', [])
        if artifacts:
            for artifact in artifacts:
                for part in artifact.get('parts', []):
                    if 'text' in part:
                        print(f"\n요약 결과:\n{part['text']}")
else:
    print("오류:", response.status_code, response.text)


## 8. ray_supply_chain_multi_agent.py


이 코드는 Ray 클러스터 환경이 필요합니다. Colab에서는 실행이 제한될 수 있으니 원본 파일을 참고해 로컬에서 실행하세요.


Ray 기반 공급망 멀티 에이전트 예제를 구성합니다.


In [ ]:
from __future__ import annotations
"""
supply_chain_logistics_agent_ray_per_session.py
세션별 격리를 통한 Ray 액터를 사용하는 멀티 에이전트 공급망 및 물류 관리 시스템을 위한 LangGraph 워크플로우.
Ray 액터로 구성된 전문 에이전트를 통해 재고 관리, 운송 작업, 공급업체 관계 및 창고 최적화를 처리합니다.
각 세션(operation_id로 식별)은 전문가 유형별로 자체 액터 인스턴스를 가지며, 세션별 격리된 상태와 순차 실행을 보장합니다.
슈퍼바이저는 전문가를 결정하고 SessionManager를 통해 세션별 액터를 원격으로 호출합니다.
실행법: 파이썬 3.12를 기준으로 생성한 venv에서 pip로 ray를 설치하고, python ch08/ray_supply_chain_multi_agent.py 명령어로 실행합니다.
"""

import os
import json
import time
from typing import Annotated, Sequence, TypedDict, Optional, Dict

import ray
from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, SystemMessage
from langchain_core.messages.tool import ToolMessage
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain_core.tools import tool
from langgraph.graph import StateGraph, END

from traceloop.sdk import Traceloop
from src.common.observability.loki_logger import log_to_loki

# 환경변수 확인
import os
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass  

if not os.getenv("OPENAI_API_KEY"):
    raise ValueError(
        "OPENAI_API_KEY가 설정되지 않았습니다."
        "환경변수 또는 .env 파일에서 설정해주세요."
    )

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "http://localhost:4317"
os.environ["OTEL_EXPORTER_OTLP_INSECURE"] = "true"

# 모든 전문가를 위한 공유 도구
@tool
def send_logistics_response(operation_id: Optional[str] = None, message: Optional[str] = None) -> str:
    """이해관계자에게 물류 업데이트, 권장 사항 또는 상태 보고서를 전송합니다."""
    print(f"[도구] send_logistics_response → {message}")
    log_to_loki("tool.send_logistics_response", f"operation_id={operation_id}, message={message}")
    return "logistics_response_sent"

# 재고 및 창고 전문가 도구
@tool
def manage_inventory(sku: Optional[str] = None, **kwargs) -> str:
    """재고 수준, 재고 보충, 감사 및 최적화 전략을 관리합니다."""
    print(f"[도구] manage_inventory(sku={sku}, kwargs={kwargs})")
    log_to_loki("tool.manage_inventory", f"sku={sku}")
    return "inventory_management_initiated"

@tool
def optimize_warehouse(operation_type: Optional[str] = None, **kwargs) -> str:
    """창고 운영, 레이아웃, 용량 및 보관 효율성을 최적화합니다."""
    print(f"[도구] optimize_warehouse(operation_type={operation_type}, kwargs={kwargs})")
    log_to_loki("tool.optimize_warehouse", f"operation_type={operation_type}")
    return "warehouse_optimization_initiated"

@tool
def forecast_demand(season: Optional[str] = None, **kwargs) -> str:
    """수요 패턴, 계절적 추세를 분석하고 예측 모델을 생성합니다."""
    print(f"[도구] forecast_demand(season={season}, kwargs={kwargs})")
    log_to_loki("tool.forecast_demand", f"season={season}")
    return "demand_forecast_generated"

@tool
def manage_quality(supplier: Optional[str] = None, **kwargs) -> str:
    """품질 관리, 결함 추적 및 공급업체 품질 표준을 관리합니다."""
    print(f"[도구] manage_quality(supplier={supplier}, kwargs={kwargs})")
    log_to_loki("tool.manage_quality", f"supplier={supplier}")
    return "quality_management_initiated"

@tool
def scale_operations(scaling_type: Optional[str] = None, **kwargs) -> str:
    """성수기, 용량 계획 및 인력 관리를 위한 운영을 확장합니다."""
    print(f"[도구] scale_operations(scaling_type={scaling_type}, kwargs={kwargs})")
    log_to_loki("tool.scale_operations", f"scaling_type={scaling_type}")
    return "operations_scaled"

@tool
def optimize_costs(cost_type: Optional[str] = None, **kwargs) -> str:
    """운송, 보관 및 운영 비용을 분석하고 최적화합니다."""
    print(f"[도구] optimize_costs(cost_type={cost_type}, kwargs={kwargs})")
    log_to_loki("tool.optimize_costs", f"cost_type={cost_type}")
    return "cost_optimization_initiated"

INVENTORY_TOOLS = [manage_inventory, optimize_warehouse, forecast_demand, manage_quality, scale_operations, optimize_costs, send_logistics_response]

# 운송 및 물류 전문가 도구
@tool
def track_shipments(origin: Optional[str] = None, **kwargs) -> str:
    """배송 상태, 지연 사항을 추적하고 배송 물류를 조정합니다."""
    print(f"[도구] track_shipments(origin={origin}, kwargs={kwargs})")
    log_to_loki("tool.track_shipments", f"origin={origin}")
    return "shipment_tracking_updated"

@tool
def arrange_shipping(shipping_type: Optional[str] = None, **kwargs) -> str:
    """배송 방법, 특급 배송 및 복합 운송을 준비합니다."""
    print(f"[도구] arrange_shipping(shipping_type={shipping_type}, kwargs={kwargs})")
    log_to_loki("tool.arrange_shipping", f"shipping_type={shipping_type}")
    return "shipping_arranged"

@tool
def coordinate_operations(operation_type: Optional[str] = None, **kwargs) -> str:
    """크로스도킹, 통합 및 이동과 같은 복잡한 작업을 조정합니다."""
    print(f"[도구] coordinate_operations(operation_type={operation_type}, kwargs={kwargs})")
    log_to_loki("tool.coordinate_operations", f"operation_type={operation_type}")
    return "operations_coordinated"

@tool
def manage_special_handling(product_type: Optional[str] = None, **kwargs) -> str:
    """위험물, 콜드체인 및 민감한 제품에 대한 특수 요구사항을 처리합니다."""
    print(f"[도구] manage_special_handling(product_type={product_type}, kwargs={kwargs})")
    log_to_loki("tool.manage_special_handling", f"product_type={product_type}")
    return "special_handling_managed"

@tool
def process_returns(returned_quantity: Optional[str] = None, **kwargs) -> str:
    """반품, 역물류 및 제품 처리를 처리합니다."""
    print(f"[도구] process_returns(returned_quantity={returned_quantity}, kwargs={kwargs})")
    log_to_loki("tool.process_returns", f"returned_quantity={returned_quantity}")
    return "returns_processed"

@tool
def optimize_delivery(delivery_type: Optional[str] = None, **kwargs) -> str:
    """배송 경로, 라스트마일 물류 및 지속가능성 이니셔티브를 최적화합니다."""
    print(f"[도구] optimize_delivery(delivery_type={delivery_type}, kwargs={kwargs})")
    log_to_loki("tool.optimize_delivery", f"delivery_type={delivery_type}")
    return "delivery_optimization_complete"

@tool
def manage_disruption(disruption_type: Optional[str] = None, **kwargs) -> str:
    """공급망 중단, 비상 계획 및 위험 완화를 관리합니다."""
    print(f"[도구] manage_disruption(disruption_type={disruption_type}, kwargs={kwargs})")
    log_to_loki("tool.manage_disruption", f"disruption_type={disruption_type}")
    return "disruption_managed"

TRANSPORTATION_TOOLS = [track_shipments, arrange_shipping, coordinate_operations, manage_special_handling, process_returns, optimize_delivery, manage_disruption, send_logistics_response]

# 공급업체 및 규정 준수 전문가 도구
@tool
def evaluate_suppliers(supplier_name: Optional[str] = None, **kwargs) -> str:
    """공급업체 성과를 평가하고 감사를 수행하며 공급업체 관계를 관리합니다."""
    print(f"[도구] evaluate_suppliers(supplier_name={supplier_name}, kwargs={kwargs})")
    log_to_loki("tool.evaluate_suppliers", f"supplier_name={supplier_name}")
    return "supplier_evaluation_complete"

@tool
def handle_compliance(compliance_type: Optional[str] = None, **kwargs) -> str:
    """규제 준수, 세관, 문서화 및 인증을 관리합니다."""
    print(f"[도구] handle_compliance(compliance_type={compliance_type}, kwargs={kwargs})")
    log_to_loki("tool.handle_compliance", f"compliance_type={compliance_type}")
    return "compliance_handled"

SUPPLIER_TOOLS = [evaluate_suppliers, handle_compliance, send_logistics_response]

# 메인 프로세스용 LLM (슈퍼바이저용)
llm = init_chat_model(model="gpt-5-mini", callbacks=[StreamingStdOutCallbackHandler()], verbose=True)

class AgentState(TypedDict):
    operation: Optional[dict]  # 공급망 운영 정보
    messages: Annotated[Sequence[BaseMessage], operator.add]

# 도구 이름으로 도구 맵 생성
TOOLS_MAP = {
    "inventory": INVENTORY_TOOLS,
    "transportation": TRANSPORTATION_TOOLS,
    "supplier": SUPPLIER_TOOLS,
}

# 전문가를 위한 Ray 액터 (세션별 격리)
# 주의: LLM 객체는 직렬화가 불가능하므로 액터 내부에서 생성해야 함
@ray.remote
class SpecialistActor:
    def __init__(self, name: str, tools_key: str, system_prompt: str):
        self.name = name
        # 액터 내부에서 LLM 초기화 (직렬화 문제 회피)
        base_llm = init_chat_model(model="gpt-5-mini", verbose=True)
        tools = TOOLS_MAP[tools_key]
        self.llm = base_llm.bind_tools(tools)
        self.tools = {t.name: t for t in tools}
        self.prompt = system_prompt
        self.internal_state = {}  # 세션별 격리된 상태, 예: 세션 내 추적용

    def process_task(self, operation: dict, messages: Sequence[BaseMessage]):
        if not operation:
            operation = {"operation_id": "알 수 없음", "type": "일반", "priority": "중간", "status": "활성"}
        operation_json = json.dumps(operation, ensure_ascii=False)
        full_prompt = self.prompt + f"\n\n작업: {operation_json}"
        
        full = [SystemMessage(content=full_prompt)] + messages

        first = self.llm.invoke(full)
        result_messages = [first]

        if hasattr(first, "tool_calls"):
            for tc in first.tool_calls:
                print(first)
                print(tc['name'])
                fn = self.tools.get(tc['name'])
                if fn:
                    out = fn.invoke(tc["args"])
                    result_messages.append(ToolMessage(content=str(out), tool_call_id=tc["id"]))

            second = self.llm.invoke(full + result_messages)
            result_messages.append(second)

        # 내부 상태 업데이트 (예: 세션 내에서 처리된 단계 추적)
        step_key = str(len(self.internal_state) + 1)  # 또는 더 구체적인 키 사용
        self.internal_state[step_key] = {"status": "처리됨", "timestamp": time.time()}

        return {"messages": result_messages}

    def get_state(self):
        return self.internal_state  # 전체 세션 상태 반환

# 세션 관리자 액터: 세션별 전문가 액터를 추적
@ray.remote
class SessionManager:
    def __init__(self):
        self.sessions: Dict[str, Dict[str, ray.actor.ActorHandle]] = {}  # session_id -> {agent_name: actor}

    def get_or_create_actor(self, session_id: str, agent_name: str, prompt: str):
        if session_id not in self.sessions:
            self.sessions[session_id] = {}
        if agent_name not in self.sessions[session_id]:
            # LLM은 액터 내부에서 생성됨 (직렬화 불가능하므로)
            actor = SpecialistActor.remote(agent_name, agent_name, prompt)
            self.sessions[session_id][agent_name] = actor
        return self.sessions[session_id][agent_name]

    def get_session_state(self, session_id: str, agent_name: str):
        if session_id in self.sessions and agent_name in self.sessions[session_id]:
            actor = self.sessions[session_id][agent_name]
            return actor.get_state.remote()  # future 반환
        return None

# 슈퍼바이저: 전문가를 결정하고 관리자를 통해 세션별 Ray 액터를 원격으로 호출
def supervisor_invoke(operation: dict, messages: Sequence[BaseMessage], manager: ray.actor.ActorHandle, prompts: dict):
    session_id = operation.get("operation_id", "알 수 없음")
    operation_json = json.dumps(operation, ensure_ascii=False)
    
    supervisor_prompt = (
        "당신은 공급망 전문가 팀을 조정하는 슈퍼바이저입니다.\n"
        "팀원:\n"
        "- inventory: 재고 수준, 예측, 품질, 창고 최적화, 확장 및 비용을 처리합니다.\n"
        "- transportation: 배송 추적, 준비, 운영 조정, 특수 처리, 반품, 배송 최적화 및 중단을 처리합니다.\n"
        "- supplier: 공급업체 평가 및 규정 준수를 처리합니다.\n"
        "\n"
        "사용자 쿼리를 기반으로 처리할 팀원 한 명을 선택하세요.\n"
        "선택한 팀원의 이름(inventory, transportation 또는 supplier)만 출력하고 다른 것은 출력하지 마세요.\n\n"
        f"작업: {operation_json}"
    )

    full = [SystemMessage(content=supervisor_prompt)] + messages
    response = llm.invoke(full)
    agent_name = response.content.strip().lower()
    
    if agent_name not in prompts:
        raise ValueError(f"알 수 없는 에이전트: {agent_name}")
    
    # 세션별 액터 가져오기 또는 생성 (LLM은 액터 내부에서 생성됨)
    actor_ref = manager.get_or_create_actor.remote(
        session_id, agent_name, prompts[agent_name]
    )
    actor = ray.get(actor_ref)  # 액터 핸들 가져오기
    
    # 원격 호출
    result_ref = actor.process_task.remote(operation, messages)
    result = ray.get(result_ref)
    return result

if __name__ == "__main__":
    Traceloop.init(disable_batch=True, app_name="supply_chain_logistics_agent_ray_per_session")
    ray.init(ignore_reinit_error=True)  # 데모용 로컬 클러스터; 분산을 위해 구성

    # 프롬프트 정의 (원본과 동일)
    inventory_prompt = (
        "당신은 재고 및 창고 관리 전문가입니다.\n"
        "관리할 때:\n"
        "  1) 재고/창고 과제를 분석합니다\n"
        "  2) 적절한 도구를 호출합니다\n"
        "  3) send_logistics_response로 후속 조치합니다\n"
        "비용, 효율성 및 확장성을 고려하세요."
    )
    transportation_prompt = (
        "당신은 운송 및 물류 전문가입니다.\n"
        "관리할 때:\n"
        "  1) 배송/전달 과제를 분석합니다\n"
        "  2) 적절한 도구를 호출합니다\n"
        "  3) send_logistics_response로 후속 조치합니다\n"
        "효율성, 지속가능성 및 위험 완화를 고려하세요."
    )
    supplier_prompt = (
        "당신은 공급업체 관계 및 규정 준수 전문가입니다.\n"
        "관리할 때:\n"
        "  1) 공급업체/규정 준수 문제를 분석합니다\n"
        "  2) 적절한 도구를 호출합니다\n"
        "  3) send_logistics_response로 후속 조치합니다\n"
        "성과, 규정 및 관계를 고려하세요."
    )

    prompts = {
        "inventory": inventory_prompt,
        "transportation": transportation_prompt,
        "supplier": supplier_prompt
    }

    # 세션 관리자 생성
    manager = SessionManager.remote()

    # 예시 호출
    example = {"operation_id": "OP-12345", "type": "inventory_management", "priority": "high", "location": "Warehouse A"}
    convo = [HumanMessage(content="SKU-12345 재고가 심각하게 부족합니다. 현재 재고는 50개이지만 미처리 주문이 200개입니다. 재주문 전략은 무엇입니까?")]

    result = supervisor_invoke(example, convo, manager, prompts)
    for m in result["messages"]:
        print(f"{m.type}: {m.content}")

    # 선택 사항: 세션별 액터 상태 쿼리
    state_ref = manager.get_session_state.remote("OP-12345", "inventory")
    if state_ref:
        state = ray.get(ray.get(state_ref))  # 중첩된 future 해결
        print("세션 액터 상태:", state)

    # Ray 종료
    ray.shutdown()


## 9. temporal_supply_chain_multi_agent.py


이 코드는 Temporal 서버/워커 구성이 필요합니다. Colab에서는 실행이 제한될 수 있으니 원본 파일을 참고해 로컬에서 실행하세요.


시간 확장 공급망 멀티 에이전트 예제를 구성합니다.


In [ ]:
from __future__ import annotations
"""
supply_chain_logistics_agent_temporal.py
내구성 있는 오케스트레이션을 위해 Temporal을 사용하는 멀티 에이전트 공급망 및 물류 관리 시스템을 위한 LangGraph 워크플로우.
Temporal 워크플로우를 통해 오케스트레이션되는 전문 에이전트를 통해 재고 관리, 운송 작업, 공급업체 관계 및 창고 최적화를 처리합니다.
워크플로우는 재시도, 영구 상태 및 장애 복구를 통해 에이전트 단계를 순차적으로 실행하며, 장기 실행 공급망 프로세스에 이상적입니다.
"""

import os
import json
from datetime import timedelta
from typing import Annotated, Sequence, TypedDict, Optional, Dict, Any

from temporalio import workflow, activity
from temporalio.common import RetryPolicy

from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, SystemMessage
from langchain_core.messages.tool import ToolMessage
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain_core.tools import tool
from temporalio.client import Client
from temporalio.worker import Worker
from temporalio.worker.workflow_sandbox import SandboxedWorkflowRunner, SandboxRestrictions, SandboxMatcher

from traceloop.sdk import Traceloop
from src.common.observability.loki_logger import log_to_loki

# 환경변수 확인
import os
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass  

if not os.getenv("OPENAI_API_KEY"):
    raise ValueError(
        "OPENAI_API_KEY가 설정되지 않았습니다."
        "환경변수 또는 .env 파일에서 설정해주세요."
    )

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "http://localhost:4317"
os.environ["OTEL_EXPORTER_OTLP_INSECURE"] = "true"

def ensure_message(m):
    if isinstance(m, BaseMessage):
        return m
    if isinstance(m, dict):
        msg_type = m.get("type")
        content = m.get("content")
        # 생성자에 전달할 kwargs에서 type 제거
        kwargs = {k:v for k,v in m.items() if k not in ["type", "content"]}
        
        if msg_type == "human":
            return HumanMessage(content=content, **kwargs)
        elif msg_type == "ai":
            return AIMessage(content=content, **kwargs)
        elif msg_type == "system":
            return SystemMessage(content=content, **kwargs)
        elif msg_type == "tool":
            return ToolMessage(content=content, **kwargs)
        return HumanMessage(content=content, **kwargs)
    return HumanMessage(content=str(m))

# 모든 전문가를 위한 공유 도구
@tool
def send_logistics_response(operation_id: Optional[str] = None, message: Optional[str] = None) -> str:
    """이해관계자에게 물류 업데이트, 권장 사항 또는 상태 보고서를 전송합니다."""
    print(f"[도구] send_logistics_response → {message}")
    log_to_loki("tool.send_logistics_response", f"operation_id={operation_id}, message={message}")
    return "logistics_response_sent"

# 재고 및 창고 전문가 도구
@tool
def manage_inventory(sku: Optional[str] = None, **kwargs) -> str:
    """재고 수준, 재고 보충, 감사 및 최적화 전략을 관리합니다."""
    print(f"[도구] manage_inventory(sku={sku}, kwargs={kwargs})")
    log_to_loki("tool.manage_inventory", f"sku={sku}")
    return "inventory_management_initiated"

@tool
def optimize_warehouse(operation_type: Optional[str] = None, **kwargs) -> str:
    """창고 운영, 레이아웃, 용량 및 보관 효율성을 최적화합니다."""
    print(f"[도구] optimize_warehouse(operation_type={operation_type}, kwargs={kwargs})")
    log_to_loki("tool.optimize_warehouse", f"operation_type={operation_type}")
    return "warehouse_optimization_initiated"

@tool
def forecast_demand(season: Optional[str] = None, **kwargs) -> str:
    """수요 패턴, 계절적 추세를 분석하고 예측 모델을 생성합니다."""
    print(f"[도구] forecast_demand(season={season}, kwargs={kwargs})")
    log_to_loki("tool.forecast_demand", f"season={season}")
    return "demand_forecast_generated"

@tool
def manage_quality(supplier: Optional[str] = None, **kwargs) -> str:
    """품질 관리, 결함 추적 및 공급업체 품질 표준을 관리합니다."""
    print(f"[도구] manage_quality(supplier={supplier}, kwargs={kwargs})")
    log_to_loki("tool.manage_quality", f"supplier={supplier}")
    return "quality_management_initiated"

@tool
def scale_operations(scaling_type: Optional[str] = None, **kwargs) -> str:
    """성수기, 용량 계획 및 인력 관리를 위한 운영을 확장합니다."""
    print(f"[도구] scale_operations(scaling_type={scaling_type}, kwargs={kwargs})")
    log_to_loki("tool.scale_operations", f"scaling_type={scaling_type}")
    return "operations_scaled"

@tool
def optimize_costs(cost_type: Optional[str] = None, **kwargs) -> str:
    """운송, 보관 및 운영 비용을 분석하고 최적화합니다."""
    print(f"[도구] optimize_costs(cost_type={cost_type}, kwargs={kwargs})")
    log_to_loki("tool.optimize_costs", f"cost_type={cost_type}")
    return "cost_optimization_initiated"

INVENTORY_TOOLS = [manage_inventory, optimize_warehouse, forecast_demand, manage_quality, scale_operations, optimize_costs, send_logistics_response]

# 운송 및 물류 전문가 도구
@tool
def track_shipments(origin: Optional[str] = None, **kwargs) -> str:
    """배송 상태, 지연 사항을 추적하고 배송 물류를 조정합니다."""
    print(f"[도구] track_shipments(origin={origin}, kwargs={kwargs})")
    log_to_loki("tool.track_shipments", f"origin={origin}")
    return "shipment_tracking_updated"

@tool
def arrange_shipping(shipping_type: Optional[str] = None, **kwargs) -> str:
    """배송 방법, 특급 배송 및 복합 운송을 준비합니다."""
    print(f"[도구] arrange_shipping(shipping_type={shipping_type}, kwargs={kwargs})")
    log_to_loki("tool.arrange_shipping", f"shipping_type={shipping_type}")
    return "shipping_arranged"

@tool
def coordinate_operations(operation_type: Optional[str] = None, **kwargs) -> str:
    """크로스도킹, 통합 및 이동과 같은 복잡한 작업을 조정합니다."""
    print(f"[도구] coordinate_operations(operation_type={operation_type}, kwargs={kwargs})")
    log_to_loki("tool.coordinate_operations", f"operation_type={operation_type}")
    return "operations_coordinated"

@tool
def manage_special_handling(product_type: Optional[str] = None, **kwargs) -> str:
    """위험물, 콜드체인 및 민감한 제품에 대한 특수 요구사항을 처리합니다."""
    print(f"[도구] manage_special_handling(product_type={product_type}, kwargs={kwargs})")
    log_to_loki("tool.manage_special_handling", f"product_type={product_type}")
    return "special_handling_managed"

@tool
def process_returns(returned_quantity: Optional[str] = None, **kwargs) -> str:
    """반품, 역물류 및 제품 처리를 처리합니다."""
    print(f"[도구] process_returns(returned_quantity={returned_quantity}, kwargs={kwargs})")
    log_to_loki("tool.process_returns", f"returned_quantity={returned_quantity}")
    return "returns_processed"

@tool
def optimize_delivery(delivery_type: Optional[str] = None, **kwargs) -> str:
    """배송 경로, 라스트마일 물류 및 지속가능성 이니셔티브를 최적화합니다."""
    print(f"[도구] optimize_delivery(delivery_type={delivery_type}, kwargs={kwargs})")
    log_to_loki("tool.optimize_delivery", f"delivery_type={delivery_type}")
    return "delivery_optimization_complete"

@tool
def manage_disruption(disruption_type: Optional[str] = None, **kwargs) -> str:
    """공급망 중단, 비상 계획 및 위험 완화를 관리합니다."""
    print(f"[도구] manage_disruption(disruption_type={disruption_type}, kwargs={kwargs})")
    log_to_loki("tool.manage_disruption", f"disruption_type={disruption_type}")
    return "disruption_managed"

TRANSPORTATION_TOOLS = [track_shipments, arrange_shipping, coordinate_operations, manage_special_handling, process_returns, optimize_delivery, manage_disruption, send_logistics_response]

# 공급업체 및 규정 준수 전문가 도구
@tool
def evaluate_suppliers(supplier_name: Optional[str] = None, **kwargs) -> str:
    """공급업체 성과를 평가하고 감사를 수행하며 공급업체 관계를 관리합니다."""
    print(f"[도구] evaluate_suppliers(supplier_name={supplier_name}, kwargs={kwargs})")
    log_to_loki("tool.evaluate_suppliers", f"supplier_name={supplier_name}")
    return "supplier_evaluation_complete"

@tool
def handle_compliance(compliance_type: Optional[str] = None, **kwargs) -> str:
    """규제 준수, 세관, 문서화 및 인증을 관리합니다."""
    print(f"[도구] handle_compliance(compliance_type={compliance_type}, kwargs={kwargs})")
    log_to_loki("tool.handle_compliance", f"compliance_type={compliance_type}")
    return "compliance_handled"

SUPPLIER_TOOLS = [evaluate_suppliers, handle_compliance, send_logistics_response]


llm = init_chat_model(model="gpt-5-mini", temperature=0.0, callbacks=[StreamingStdOutCallbackHandler()], verbose=True)

# 전문화된 LLM에 도구 바인딩
inventory_llm = llm.bind_tools(INVENTORY_TOOLS)
transportation_llm = llm.bind_tools(TRANSPORTATION_TOOLS)
supplier_llm = llm.bind_tools(SUPPLIER_TOOLS)

class AgentState(TypedDict):
    operation: Optional[dict]  # 공급망 운영 정보
    messages: Annotated[Sequence[BaseMessage], "add"]

# Temporal 액티비티 (전문가 로직 래핑)
@activity.defn
async def supervisor_activity(operation: Dict[str, Any], messages: list) -> Dict[str, Any]:
    """슈퍼바이저를 통해 전문가를 결정하는 액티비티."""
    operation_json = json.dumps(operation, ensure_ascii=False)
    
    supervisor_prompt = (
        "당신은 공급망 전문가 팀을 조정하는 슈퍼바이저입니다.\n"
        "팀원:\n"
        "- inventory: 재고 수준, 예측, 품질, 창고 최적화, 확장 및 비용을 처리합니다.\n"
        "- transportation: 배송 추적, 준비, 운영 조정, 특수 처리, 반품, 배송 최적화 및 중단을 처리합니다.\n"
        "- supplier: 공급업체 평가 및 규정 준수를 처리합니다.\n"
        "\n"
        "사용자 쿼리를 기반으로 처리할 팀원 한 명을 선택하세요.\n"
        "선택한 팀원의 이름(inventory, transportation 또는 supplier)만 출력하고 다른 것은 출력하지 마세요.\n\n"
        f"작업: {operation_json}"
    )

    full = [SystemMessage(content=supervisor_prompt)] + [ensure_message(m) for m in messages]
    response = llm.invoke(full)
    agent_name = response.content.strip().lower()
    return {"agent_name": agent_name, "messages": [response.dict()]}

@activity.defn
async def specialist_activity(agent_name: str, operation: Dict[str, Any], messages: list) -> Dict[str, Any]:
    """전문가 처리를 위한 액티비티 (inventory, transportation, supplier)."""
    if agent_name not in prompts:
        raise ValueError(f"알 수 없는 에이전트: {agent_name}")
    
    specialist_llm = llms_dict[agent_name]
    tools = {t.name: t for t in tools_dict[agent_name]}
    system_prompt = prompts[agent_name]
    
    operation_json = json.dumps(operation, ensure_ascii=False)
    full_prompt = system_prompt + f"\n\n작업: {operation_json}"
    
    full = [SystemMessage(content=full_prompt)] + [ensure_message(m) for m in messages]

    first = specialist_llm.invoke(full)
    result_messages = [first.dict()]

    if hasattr(first, "tool_calls"):
        for tc in first.tool_calls:
            fn = tools.get(tc['name'])
            if fn:
                out = fn.invoke(tc["args"])
                result_messages.append(ToolMessage(content=str(out), tool_call_id=tc["id"]).dict())

        second = specialist_llm.invoke(full + [ensure_message(msg) for msg in result_messages])
        result_messages.append(second.dict())

    return {"messages": result_messages}

# Temporal 워크플로우
@workflow.defn(name="SupplyChainWorkflow")
class SupplyChainWorkflow:
    @workflow.run
    async def run(self, operation: Dict[str, Any], initial_messages: list) -> Dict[str, Any]:
        # 단계 1: 슈퍼바이저가 라우팅
        supervisor_result = await workflow.execute_activity(
            supervisor_activity,
            args=[operation, initial_messages],
            start_to_close_timeout=timedelta(seconds=60),
            retry_policy=RetryPolicy(maximum_attempts=3)
        )
        agent_name = supervisor_result["agent_name"]
        updated_messages = initial_messages + supervisor_result["messages"]
        
        # 단계 2: 전문가 처리
        specialist_result = await workflow.execute_activity(
            specialist_activity,
            args=[agent_name, operation, updated_messages],
            start_to_close_timeout=timedelta(seconds=60),
            retry_policy=RetryPolicy(maximum_attempts=3)
        )
        
        # 결과 컴파일 (필요시 다단계로 확장)
        final_messages = updated_messages + specialist_result["messages"]
        return {
            "agent_name": agent_name,
            "final_messages": final_messages,
            "operation": operation
        }

# 프롬프트 정의
inventory_prompt = (
    "당신은 재고 및 창고 관리 전문가입니다.\n"
    "관리할 때:\n"
    "  1) 재고/창고 과제를 분석합니다\n"
    "  2) 적절한 도구를 호출합니다\n"
    "  3) send_logistics_response로 후속 조치합니다\n"
    "비용, 효율성 및 확장성을 고려하세요."
)

transportation_prompt = (
    "당신은 운송 및 물류 전문가입니다.\n"
    "관리할 때:\n"
    "  1) 배송/전달 과제를 분석합니다\n"
    "  2) 적절한 도구를 호출합니다\n"
    "  3) send_logistics_response로 후속 조치합니다\n"
    "효율성, 지속가능성 및 위험 완화를 고려하세요."
)

supplier_prompt = (
    "당신은 공급업체 관계 및 규정 준수 전문가입니다.\n"
    "관리할 때:\n"
    "  1) 공급업체/규정 준수 문제를 분석합니다\n"
    "  2) 적절한 도구를 호출합니다\n"
    "  3) send_logistics_response로 후속 조치합니다\n"
    "성과, 규정 및 관계를 고려하세요."
)

prompts = {
    "inventory": inventory_prompt,
    "transportation": transportation_prompt,
    "supplier": supplier_prompt
}

llms_dict = {
    "inventory": inventory_llm,
    "transportation": transportation_llm,
    "supplier": supplier_llm
}

tools_dict = {
    "inventory": INVENTORY_TOOLS,
    "transportation": TRANSPORTATION_TOOLS,
    "supplier": SUPPLIER_TOOLS
}

async def main():
    client = await Client.connect("localhost:7233")
    
    # 샌드박스 설정: LangChain 등 외부 라이브러리 허용
    runner = SandboxedWorkflowRunner(
        restrictions=SandboxRestrictions(
            invalid_modules=SandboxMatcher(),
            invalid_module_members=SandboxRestrictions.invalid_module_members_default,
            passthrough_modules=SandboxRestrictions.passthrough_modules_default | {
                "langchain", 
                "langchain_core", 
                "langchain_community", 
                "langchain_openai",
                "pydantic", 
                "requests", 
                "urllib3", 
                "http",
                "certifi",
                "charset_normalizer",
                "idna",
                "ssl",
                "socket",
                "logging",
                "tenacity",
                "traceloop"
            }
        )
    )

    # 워커 시작
    async with Worker(
        client, 
        task_queue="supply-chain-queue", 
        workflows=[SupplyChainWorkflow], 
        activities=[supervisor_activity, specialist_activity],
        workflow_runner=runner
    ):
        # 예시 실행
        example_operation = {"operation_id": "OP-12345", "type": "inventory_management", "priority": "high", "location": "Warehouse A"}
        example_messages = [{"content": "SKU-12345 재고가 심각하게 부족합니다. 현재 재고는 50개이지만 미처리 주문이 200개입니다. 재주문 전략은 무엇입니까?", "type": "human"}]

        result = await client.execute_workflow(
            SupplyChainWorkflow.run,
            args=[example_operation, example_messages],
            id="supply-chain-workflow",
            task_queue="supply-chain-queue"
        )
        print("워크플로우 결과:")
        for m in result["final_messages"]:
            print(m)

if __name__ == "__main__":
    import asyncio
    Traceloop.init(disable_batch=True, app_name="supply_chain_logistics_agent_temporal")
    asyncio.run(main())
